In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [3]:
import pandas as pd
from dataiku import Dataset
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# --- jeu de données
df = Dataset("annual_default_rates_sf").get_dataframe()
df = df.dropna(subset=["taux", "region", "sector", "ANNEE"])

# --- encodage simple
X = pd.get_dummies(df[["region", "sector", "ANNEE"]], drop_first=True)
y = df["taux"]

# --- séparation apprentissage / validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- modèle
rf = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

# --- validation
y_pred = rf.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"R²={r2:.3f}  RMSE={rmse:.4f}")

# --- score de risque
score = 100 * (y_pred / y_pred.max())
score = np.clip(score, 0, 100)

# --- tableau des résultats
out = df.loc[y_test.index, ["region", "sector", "ANNEE"]].copy()
out["taux_reel"] = y_test.values
out["taux_pred"] = y_pred
out["score_risque"] = score
out = out.sort_values("score_risque", ascending=False).reset_index(drop=True)

print("\nsegments les plus risqués :")
print(out.head(10))


R²=0.960  RMSE=0.0209

segments les plus risqués :
                 region                       sector  ANNEE  taux_reel  taux_pred  score_risque
0         Île-de-France                 Construction   2016   0.407057   0.383708    100.000000
1       Hauts-de-France  Hébergement et restauration   2017   0.376643   0.369629     96.330992
2      Pays de la Loire  Hébergement et restauration   2014   0.321557   0.366244     95.448651
3       Hauts-de-France                 Construction   2014   0.412502   0.350296     91.292419
4         Île-de-France                 Construction   2017   0.373639   0.343033     89.399526
5             Grand Est  Hébergement et restauration   2017   0.316263   0.342516     89.264954
6       Hauts-de-France  Hébergement et restauration   2019   0.341056   0.336257     87.633756
7  Auvergne-Rhône-Alpes  Hébergement et restauration   2014   0.287439   0.336126     87.599484
8   Centre-Val de Loire  Hébergement et restauration   2015   0.398376   0.330956    

/opt/dataiku/pyenv/lib64/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [5]:
def compute_risk(region, sector, year):
    """
    Retourne le score de risque (0–100) prédit pour une combinaison donnée.
    """
    # crée un petit dataframe pour la prédiction
    X_new = pd.DataFrame([{"region": region, "sector": sector, "ANNEE": year}])
    X_new = pd.get_dummies(X_new, drop_first=True)
    # aligne les colonnes sur celles du modèle
    X_new = X_new.reindex(columns=X.columns, fill_value=0)
    
    # prédiction du taux
    taux_pred = rf.predict(X_new)[0]
    
    # normalisation en score 0–100
    max_ref = y_pred.max() if 'y_pred' in locals() else taux_pred
    score = 100 * (taux_pred / max_ref)
    score = float(np.clip(score, 0, 100))
    
    return {
        "region": region,
        "sector": sector,
        "ANNEE": year,
        "taux_pred": round(float(taux_pred), 4),
        "score_risque": round(score, 1)
    }



In [12]:
import pandas as pd
import numpy as np

# listes
regions = [
 'Hauts-de-France','Bretagne','Normandie','Grand Est','Pays de la Loire',
 'Centre-Val de Loire','Bourgogne-Franche-Comté','Nouvelle-Aquitaine',
 'Auvergne-Rhône-Alpes','Occitanie',"Provence-Alpes-Côte d'Azur",
 'Corse','Île-de-France'
]

sectors = [
 'Construction','Transports et entreposage','Hébergement et restauration',
 'Information et communication',"Activités financières et d'assurance",
 'Activités immobilières'
]

reg_code = {
 'Île-de-France':'11','Centre-Val de Loire':'24','Bourgogne-Franche-Comté':'27',
 'Normandie':'28','Hauts-de-France':'32','Grand Est':'44','Pays de la Loire':'52',
 'Bretagne':'53','Nouvelle-Aquitaine':'75','Occitanie':'76','Auvergne-Rhône-Alpes':'84',
 "Provence-Alpes-Côte d'Azur":'93','Corse':'94'
}

a21_code = {
 'Construction':'F','Transports et entreposage':'H','Hébergement et restauration':'I',
 'Information et communication':'J',"Activités financières et d'assurance":'K',
 'Activités immobilières':'L'
}

# toutes les combinaisons possibles pour 2026
rows = []
for r in regions:
    for s in sectors:
        rows.append({"region": r, "sector": s, "ANNEE": 2026,
                     "REG": reg_code[r], "A21": a21_code[s]})

df_2026 = pd.DataFrame(rows)

# encodage des variables comme dans le modèle
X_2026 = pd.get_dummies(df_2026[["region","sector","ANNEE"]], drop_first=False)
X_2026 = X_2026.reindex(columns=X.columns, fill_value=0)

# prédiction et score
pred = rf.predict(X_2026)
p95 = np.percentile(rf.predict(X), 95)
score = 100 * pred / p95
score = np.clip(score, 0, 100)

df_2026["taux_pred"] = pred.round(4)
df_2026["score_risque"] = score.round(1)
df_2026 = df_2026.sort_values("score_risque", ascending=False).reset_index(drop=True)

print(df_2026.head(15))

from dataiku import Dataset
Dataset("RISK_SCORE_2026").write_with_schema(df_2026)


                        region                       sector  ANNEE REG A21  taux_pred  score_risque
0                Île-de-France                 Construction   2026  11   F     0.1409          42.3
1              Hauts-de-France                 Construction   2026  32   F     0.1147          34.5
2              Hauts-de-France  Hébergement et restauration   2026  32   I     0.0997          29.9
3          Centre-Val de Loire                 Construction   2026  24   F     0.0986          29.6
4                    Grand Est                 Construction   2026  44   F     0.0968          29.1
5          Centre-Val de Loire  Hébergement et restauration   2026  24   I     0.0939          28.2
6                    Grand Est  Hébergement et restauration   2026  44   I     0.0927          27.8
7                Île-de-France  Hébergement et restauration   2026  11   I     0.0900          27.0
8      Bourgogne-Franche-Comté  Hébergement et restauration   2026  27   I     0.0897          26.9


/opt/dataiku/python/dataiku/core/schema_handling.py:68: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if is_datetime64tz_dtype(dtype):


78 rows successfully written (3w15QYEd6l)
